In [9]:
from loader import load_strips
from panel_splitter import panelizer
import cv2
import ollama

In [7]:
prompt = """Role: You are a specialized OCR transcription engine for comics.

Context: This is a comic panel.

Dilbert is the guy wearing a red and black tie.
Dogbert is the white dog.
For any other character whose name is unknown just say unknown.

Task: Transcribe ONLY the dialogue spoken by characters.

Rules:
Do NOT summarize the plot. If nothing is spoken in the whole image say '<No conversation>'
Do NOT describe the characters' emotions (e.g., "grumpy").
Only pick up text in speech bubbles.
Determine which character said the text by following where the tail near the text points to.

Output Format:
[Character]: [Text]

Image Analysis:"""

In [ ]:
strips = load_strips("dilbert_1989_to_2023")
for strip in strips:
    print('\n\n_________________________________________\nDate:', strip['date'])
    with open("llava_processed.txt", "a", encoding="utf-8") as f:
        f.write('\n\n_________________________________________\nDate: ' + strip['date'] + '\n')
    image = strip["image"]
    panels = panelizer(strip)
    #resized_image = cv2.resize(image, None, fx=0.8, fy=0.8, interpolation=cv2.INTER_AREA)
    for i, panel in enumerate(panels):
        print("\nPanel", str(i+1) + ':')
        _, buffer = cv2.imencode(".png", panel)
        response = ollama.chat(
        model='llava:13b',
        #model='llama3.2-vision',
        #model='blaifa/InternVL3_5:8b',
        options={'temperature': 0},
        messages=[{
            'role': 'user',
            'content': prompt,
            'images': [buffer.tobytes()]
        }]
        )

        print(response['message']['content'])
        with open("llava_processed.txt", "a", encoding="utf-8") as f:
            f.write('\nPanel ' + str(i+1) + ':\n' + response['message']['content'] + '\n')



_________________________________________
Date: 1990-01-01

Panel 1:
 Dilbert: I'm grumpy today, so don't even try to talk to me. . . So don't even try to talk to me.

Panel 2:
 Dogbert: And don't try to flatter me or give me chocolate cake to make me feel better.

Panel 3:
 Dilbert: And I guess, I shouldn't scratch you behind the ears until you have little leg spasms. Right? None of that stuff.


_________________________________________
Date: 1990-01-02

Panel 1:
 Dilbert: I'm starting to write an unauthorized biography about you.

Panel 2:
 Dogbert: It's kind of a "pet" and tell "full of staring revelations."

Panel 3:
 Dilbert: Who would be started by my life?
Dogbert: I think you will be.


_________________________________________
Date: 1990-01-03

Panel 1:
 Dilbert: Are you really going through with the unauthorized biography of me?
Dogbert: Yes.

Panel 2:
 Dogbert: I'm up to the part where Jackie O and Liz Taylor fight a duel for your love.

Panel 3:
 Dilbert: Tap tap tap!
Do